# LSTM Series (working)

** - Matlab initialization strategy **

** - Train and Dev different loss functions **

** - Adam Optimizer **
_Imag only_

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import math

import scipy.io as sio
import numpy as np
import h5py
import tensorflow as tf
import sys

import time

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"


from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, TimeDistributed
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


### Read parameters from .mat files
save in Opt object

In [2]:
Test_Data_VERSION = "_e10v5"
Code_VERSION = "_e11v9-2"

In [3]:
DNN_DATA_FILE = "./dnn_models/Test_datas"+ Test_Data_VERSION+".mat"

# MODEL_FILE = "./dnn_models/py_model"+ Code_VERSION+".h5"
MODEL_FILE = "./dnn_models/weights_15_e11v9-2.h5"

OUTPUT_FILE = "./dnn_models/Real_Imag"+ Code_VERSION+".mat"

Files saved by Matlab reading class. Data, Parameters are read. 

In [4]:
class Opts:
    opts_dict = dict()

    def __init__(self, FILE_DATA):

        # Testing Data 
        with h5py.File(FILE_DATA, 'r') as f:
            print('Opts h5py keys:', list(f.keys()))
            for k, v in f.items():

                if k == 'teData':
                    print("teData.shape: ", v.shape)
                    self.teData = np.transpose(np.array(v))
                    print("teData-> mean:", np.mean(self.teData), ", var:", np.var(self.teData), ", std:",
                          np.std(self.teData), ", range:", (np.amin(self.teData),np.amax(self.teData)))
                    

                elif k == 'teLabel_i':
                    print("teLabel_i.shape: ", v.shape)
                    self.teLabel_i = np.transpose(np.array(v))
                    print("teLabel_i-> mean:", np.mean(self.teLabel_i), ", var:", np.var(self.teLabel_i), ", std:",
                          np.std(self.teLabel_i), ", range:", (np.amin(self.teLabel_i),np.amax(self.teLabel_i)))
                    
                elif k == 'teLabel_r':
                    print("teLabel_r.shape: ", v.shape)
                    self.teLabel_r = np.transpose(np.array(v))
                    print("teLabel_r-> mean:", np.mean(self.teLabel_r), ", var:", np.var(self.teLabel_r), ", std:",
                          np.std(self.teLabel_r), ", range:", (np.amin(self.teLabel_r),np.amax(self.teLabel_r)))
                
                elif k == 'teNumframes':
                    print("teNumframes.shape: ", v.shape)
                    self.teNumframes = np.transpose(np.array(v))
                    
            self.teLabel = np.concatenate((self.teLabel_r, self.teLabel_i), axis=1)

     
    def prepare3D_list(self):
        # TEST: total_num_samples = self.teData.shape[0] = 72440
        
        feat_vec_len = 1230
        out_vec_len = 963
        
        data = self.teData
        label_r = self.teLabel_r
        label_i = self.teLabel_i
        numframes = self.teNumframes

        data3D, label3D_r, label3D_i = [], [], []
        numframes = np.cumsum(numframes)
        
        for e, frames in enumerate(numframes):
            frames= int(frames)
            pre_frames= int(numframes[e-1])
            
            d = data[:frames] if len(data3D)==0 else data[pre_frames:frames]
            r = label_r[:frames] if len(label3D_r)==0 else label_r[pre_frames:frames]
            i = label_i[:frames] if len(label3D_i)==0 else label_i[pre_frames:frames]
            
            data3D.append( d )
            label3D_r.append( r )
            label3D_i.append( i )

        return data3D, label3D_r, label3D_i


    def ready_batchID(self, total_num_samples, batch_size):
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        batchID = []
        num_batch = math.ceil(total_num_samples/batch_size)

        for b in range( int(num_batch) ):
            s = b*batch_size
            e = (b+1)*batch_size -1

            if e >= total_num_samples:
                e = total_num_samples - 1

            batchID.append((s,e))

        return np.array(batchID,ndmin=2)


    def suffle_data(self, total_num_samples):
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        return  np.random.permutation(total_num_samples)


    def next_batch(self, total_num_samples, batch_size, isTrainCycle=True):
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        batchID = self.ready_batchID(total_num_samples, batch_size)
        seq = self.suffle_data(total_num_samples)

        for batch in range(batchID.shape[0]):
            if isTrainCycle:
                x = opts.trData[ seq[batchID[batch][0]:batchID[batch][1] ] ]
                y = opts.trLabel[ seq[batchID[batch][0]:batchID[batch][1] ] ]

            else:
                x = opts.cvData[seq[batchID[batch][0]:batchID[batch][1]]]
                y = opts.cvLabel[seq[batchID[batch][0]:batchID[batch][1]]]

            # print('Next Batch', x.shape, y.shape)
            yield [x, y]
            

In [5]:
opts = Opts(DNN_DATA_FILE)

Opts h5py keys: ['#refs#', '#subsystem#', 'teData', 'teFilename_mix', 'teLabel_i', 'teLabel_r', 'teNumframes']
teData.shape:  (1230, 72440)
teData-> mean: -0.00110349 , var: 0.556445 , std: 0.745952 , range: (-4.8041673, 5.2418036)
teLabel_i.shape:  (963, 72440)
teLabel_i-> mean: -3.01466e-06 , var: 1.05955e-05 , std: 0.00325507 , range: (-0.03745788, 0.035189673)
teLabel_r.shape:  (963, 72440)
teLabel_r-> mean: 0.0328164 , var: 0.00246607 , std: 0.0496596 , range: (-0.15398552, 0.43288523)
teNumframes.shape:  (1, 545)


opts.teFilename_mix

In [ ]:
opts

### Batch generator (now)

In [6]:
def test_batch_rnn(opts, maxlen=185):
    # TEST: total_num_samples = self.trData.shape[0] = 72440

    feat_vec_len = 1230
    out_vec_len = 963

    selected_indics = np.arange(len(opts.teNumframes))
    data3D, label3D_r, label3D_i = opts.prepare3D_list()

    f = 0
    x, y = None, None
    
    for indx in selected_indics:
        d = data3D[indx]
        d = np.concatenate( (d, np.zeros((maxlen-d.shape[0],feat_vec_len))),axis=0 )
        d = np.expand_dims(d, axis=0)

        l = label3D_r[indx] # here real only
        l = np.concatenate( (l, np.zeros((maxlen-l.shape[0],out_vec_len))),axis=0 )
        l = np.expand_dims(l, axis=0)

        x = d if x is None else np.concatenate( (x,d),axis=0 )
        y = l if y is None else np.concatenate( (y,l),axis=0 )

    f += 1

#                 if x.shape[0]<batch_size:
#                     x = np.concatenate( (x, np.zeros((batch_size-x.shape[0],maxlen,feat_vec_len))), axis=0)
#                     y = np.concatenate( (y, np.zeros((batch_size-y.shape[0],maxlen,out_vec_len*2))), axis=0)

    print("x:",x.shape, ", y:", y.shape)
    return x,y
                        

In [8]:
x,y_r = test_batch_rnn(opts, maxlen=Max_RNN) 

x: (545, 200, 1230) , y: (545, 200, 963)


### RNN Testing
single bidirectional GRU layer

real+img (963+963)=1926-d output vector

In [7]:
Max_RNN = 200# 185

feat_vec_len = 1230
out_vec_len = 963

In [9]:
model = load_model(MODEL_FILE)


y_hat = model.predict(x, batch_size=x.shape[0], verbose=1)

print(y_hat.shape)

545/545 [==============================] - 2s
(545, 185, 963)


### For Stateful

In [9]:
batch_size = 32

if x.shape[0]%batch_size:
    d = x.shape[0]//batch_size + 1
    
    x = np.concatenate((x, np.zeros((batch_size*d-x.shape[0],x.shape[1],x.shape[2]))), axis=0)
    y_r = np.concatenate((y_r, np.zeros((batch_size*d-y_r.shape[0],y_r.shape[1],y_r.shape[2]))), axis=0)
    
print(x.shape, y_r.shape)

(576, 200, 1230) (576, 200, 963)


In [10]:
model = load_model(MODEL_FILE)
for b in range(d):
    if b==0:
        y_hat = model.predict(x[batch_size*b:batch_size*(b+1)], batch_size=batch_size, verbose=1)
    elif b==d: 
        y_hat = np.concatenate( \
            (y_hat,model.predict(x[batch_size*b:], batch_size=batch_size, verbose=1))\
            ,axis=0)
    else: 
        y_hat = np.concatenate( \
            (y_hat,model.predict(x[batch_size*b:batch_size*(b+1)], batch_size=batch_size, verbose=1))\
            ,axis=0)

print(y_hat.shape)

32/32 [==============================] - 0s
(576, 200, 963)


In [11]:
np.concatenate((y_r, y_hat), axis=2).shape

(576, 200, 1926)

In [13]:
sio.savemat(OUTPUT_FILE, {'y_hat':np.concatenate((y_hat, y_r), axis=2)[:545]}) # originally 545 (not 576)

### unwrapAugmentedTF_wAvg()

In [10]:
def unwrapAugmentedTF_wAvg(impt_mag,num_per_side,unwrap_mag=None):
    
#     Description: Unwrap the augmented time-frequency (T-F) representation and compute the average.

#     Input:
#         impt_mag: wrapped T-F representation with dimensions (2*T+1)*d x m
#         T: number of frames to left and right of each frame used to augmented T-F representation
#     Output:
#         unwrap_avgmag: unwrapped and averaged T-F representation with dimensions d x m
#         unwrap_mag: unwrapped T-F representation with dimensions d x (2*T+1) x m
#
        
        sliding_window_len       = 2*num_per_side + 1
        (numWrapFreqs,numFrames) = impt_mag.shape
        numFreqs                 = numWrapFreqs//sliding_window_len
        unwrap_avgmag            = np.zeros((numFreqs,numFrames))
        
        if num_per_side > 0:
            if unwrap_mag is None:
            
                unwrap_mag        = np.zeros((numFreqs,sliding_window_len+1,numFrames))
                curr_ind_location = np.ones((numFrames,1),dtype=int)
                

                for frameNum in range(numFrames):
                    
                    # Get the indices for the frames used in this augmented matrix
                    frame_inds=[]
                    for inds in range(frameNum-num_per_side,frameNum+num_per_side+1):
                        if inds<0:
                            frame_inds.append(0)
                        elif inds>=numFrames:
                            frame_inds.append(numFrames-1)
                        else:
                            frame_inds.append(inds)
                    
                    # Unwrap the data for this frame
                    slid_win_data = np.reshape( impt_mag[:,frameNum], (numFreqs,sliding_window_len)) #Size d x (2*T + 1)
                    
                    for ind_num in range(len(frame_inds)):

                        slid = np.array(slid_win_data[:,ind_num], ndmin=2).T
                        unwrap_mag[:,curr_ind_location[frame_inds[ind_num]], frame_inds[ind_num]] = slid 
                        
                        # Update counters
                        curr_ind_location[frame_inds[ind_num]] = curr_ind_location[frame_inds[ind_num]] + 1


            temp = np.zeros((numFreqs,sliding_window_len+1))
            
            for frameNum in range(numFrames):
#                 print(unwrap_mag.shape)
                
                unwrap_avgmag[:,frameNum] = np.mean(unwrap_mag[:,:,frameNum],axis=1)
#                 print(unwrap_mag.shape)
#                 return

            
        else:
            unwrap_avgmag = impt_mag;
            unwrap_mag = 0;
            
        return unwrap_avgmag,unwrap_mag

In [12]:
reals=[]
imags=None

labwin = 1
for e, y_ in enumerate(y_hat):
    
    numFrames = int(opts.teNumframes[e])
    
    real_output = y_[:numFrames,:out_vec_len]
    imag_output = y_[:numFrames,out_vec_len:]
#     print(e, numFrames, real_output.shape, imag_output.shape)
    
    real_output_unwrap, _ = unwrapAugmentedTF_wAvg(real_output.T,labwin)
    imag_output_unwrap, _ = unwrapAugmentedTF_wAvg(imag_output.T,labwin)
    
    print('unwrap:',real_output_unwrap.shape, end=' , ')
    
#     reals= np.array([list(real_output_unwrap)], dtype=object) if reals is None else np.append(reals,[list(real_output_unwrap)], axis=0)
    reals.append(real_output_unwrap)
    print('reals:',len(reals))
#     imags.append(imag_output_unwrap)
    

unwrap: (321, 127) , reals: 1
unwrap: (321, 127) , reals: 2
unwrap: (321, 127) , reals: 3
unwrap: (321, 127) , reals: 4
unwrap: (321, 127) , reals: 5
unwrap: (321, 147) , reals: 6
unwrap: (321, 147) , reals: 7
unwrap: (321, 147) , reals: 8
unwrap: (321, 147) , reals: 9
unwrap: (321, 147) , reals: 10
unwrap: (321, 131) , reals: 11
unwrap: (321, 131) , reals: 12
unwrap: (321, 131) , reals: 13
unwrap: (321, 131) , reals: 14
unwrap: (321, 131) , reals: 15
unwrap: (321, 147) , reals: 16
unwrap: (321, 147) , reals: 17
unwrap: (321, 147) , reals: 18
unwrap: (321, 147) , reals: 19
unwrap: (321, 147) , reals: 20
unwrap: (321, 145) , reals: 21
unwrap: (321, 145) , reals: 22
unwrap: (321, 145) , reals: 23
unwrap: (321, 145) , reals: 24
unwrap: (321, 145) , reals: 25
unwrap: (321, 141) , reals: 26
unwrap: (321, 141) , reals: 27
unwrap: (321, 141) , reals: 28
unwrap: (321, 141) , reals: 29
unwrap: (321, 141) , reals: 30
unwrap: (321, 155) , reals: 31
unwrap: (321, 155) , reals: 32
unwrap: (321, 155

unwrap: (321, 120) , reals: 270
unwrap: (321, 136) , reals: 271
unwrap: (321, 136) , reals: 272
unwrap: (321, 136) , reals: 273
unwrap: (321, 136) , reals: 274
unwrap: (321, 136) , reals: 275
unwrap: (321, 123) , reals: 276
unwrap: (321, 123) , reals: 277
unwrap: (321, 123) , reals: 278
unwrap: (321, 123) , reals: 279
unwrap: (321, 123) , reals: 280
unwrap: (321, 145) , reals: 281
unwrap: (321, 145) , reals: 282
unwrap: (321, 145) , reals: 283
unwrap: (321, 145) , reals: 284
unwrap: (321, 145) , reals: 285
unwrap: (321, 123) , reals: 286
unwrap: (321, 123) , reals: 287
unwrap: (321, 123) , reals: 288
unwrap: (321, 123) , reals: 289
unwrap: (321, 123) , reals: 290
unwrap: (321, 126) , reals: 291
unwrap: (321, 126) , reals: 292
unwrap: (321, 126) , reals: 293
unwrap: (321, 126) , reals: 294
unwrap: (321, 126) , reals: 295
unwrap: (321, 117) , reals: 296
unwrap: (321, 117) , reals: 297
unwrap: (321, 117) , reals: 298
unwrap: (321, 117) , reals: 299
unwrap: (321, 117) , reals: 300
unwrap: 

unwrap: (321, 141) , reals: 529
unwrap: (321, 141) , reals: 530
unwrap: (321, 130) , reals: 531
unwrap: (321, 130) , reals: 532
unwrap: (321, 130) , reals: 533
unwrap: (321, 130) , reals: 534
unwrap: (321, 130) , reals: 535
unwrap: (321, 112) , reals: 536
unwrap: (321, 112) , reals: 537
unwrap: (321, 112) , reals: 538
unwrap: (321, 112) , reals: 539
unwrap: (321, 112) , reals: 540
unwrap: (321, 105) , reals: 541
unwrap: (321, 105) , reals: 542
unwrap: (321, 105) , reals: 543
unwrap: (321, 105) , reals: 544
unwrap: (321, 105) , reals: 545


In [17]:
np.savetxt('./dnn_models/test.out', reals[0], delimiter=',')   # X is an array

In [ ]:
def write_file(FILE_Name, *argv):
    
    saved_list=[]
    name_list=[]
    dic = dict()
    
    for arg in argv:
        if isinstance(arg, str):
            name_list.append(arg)
        else:
            saved_list.append(arg)
        
    for n in name_list:
        print(type(n), type(saved_list.pop(0)))
#         dic[n]=np.array(saved_list.pop(0),  dtype=np.object)
        
#     Param_Dict = np.core.records.fromarrays( saved_list, names=",".join(name_list))
#     sio.savemat(FILE_Name, Param_Dict, format='5', long_field_names=True)
#     sio.savemat(FILE_Name, dic)
 

In [34]:
write_file(OUTPUT_FILE,'real_output_unwrap',reals,'imag_output_unwrap', imags)

<class 'str'> <class 'list'>
<class 'str'> <class 'list'>


In [28]:
len(reals)

545

### Parameters

In [ ]:
## HAVE to use OPS PARAMS ***********
learning_rate = 0.001
# training_epochs = 80
# batch_size = 256
display_step = 1

### Network Parameters

In [ ]:
## HAVE to use NET_STRUCTURE ***********
n_input_dim = 195192.0
n_input = 1230  # data input
n_hidden_1 = 1024  # 1st layer number of neurons
n_hidden_2 = 1024  # 2nd layer number of neurons
n_hidden_3 = 1024  # 3rd layer number of neurons
n_classes = (963 + 963)  # total classes (real+imaginary)

# For checking
# train_time = np.zeros(training_epochs)

# validation_error = np.full((1), np.inf)
# min_validation_error = np.full((1), np.inf)

Best_Cost = - np.inf
Best_Weight, Best_Bias = None, None
Best_epoch = -1

# For displaying
PREVIOUS_10 = 10
DIFF_THRESHOLD = 0.000001


### Fully-Connected 3 layer Feed Forward Net

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, n_input])
Y = tf.placeholder(tf.float32, shape=[None, n_classes])


# Xavier Initialization
def weight_variable(shape, stddev=None, stddev_2=None):
    if stddev is None:
        initial = tf.truncated_normal(shape, stddev=np.sqrt(2.0 / sum(shape)))
    elif stddev > 0.0:
        if stddev_2 is not None:
            r, c = shape
            initial1 = tf.truncated_normal([r, c//2], stddev=stddev)
            initial2 = tf.truncated_normal([r, c//2], stddev=stddev_2)
            initial = tf.concat([initial1,initial2], axis=1)
        else:
            initial = tf.truncated_normal(shape, stddev=stddev)

    else:
        initial = tf.constant(0.0, shape=shape)

    return tf.Variable(initial)


def bias_variable(shape, stddev=None, stddev_2=None):
    # initial = tf.constant(0.1, shape=shape)
    if stddev is None:
        initial = tf.truncated_normal(shape, stddev=np.sqrt(1.0 / sum(shape)))
    elif stddev > 0.0:
        if stddev_2 is not None:
            r, c = shape
            initial1 = tf.truncated_normal([r, c//2], stddev=stddev)
            initial2 = tf.truncated_normal([r, c//2], stddev=stddev_2)
            initial = tf.concat([initial1,initial2], axis=1)
        else:
            initial = tf.truncated_normal(shape, stddev=stddev)

    else:
        initial = tf.constant(0.0, shape=shape)

    return tf.Variable(initial)

In [ ]:
# old way
weights = {
    'h1': weight_variable([n_input, n_hidden_1],0.001),
    'h2': weight_variable([n_hidden_1, n_hidden_2],0.001),
    'h3': weight_variable([n_hidden_2, n_hidden_3],0.001),
    'out': weight_variable([n_hidden_3, n_classes],0.001,0.0)
}
biases = {
    'b1': bias_variable([1, n_hidden_1],0.0),
    'b2': bias_variable([1, n_hidden_2],0.0),
    'b3': bias_variable([1, n_hidden_3],0.0),
    'out': bias_variable([1, n_classes],0.001,0.0)
}

### Create model

In [ ]:
def multilayer_NN(x):
    layer_1 = tf.nn.relu(tf.matmul(x, weights['h1']) + biases['b1'])
    layer_2 = tf.nn.relu(tf.matmul(layer_1, weights['h2']) + biases['b2'])
    layer_3 = tf.nn.relu(tf.matmul(layer_2, weights['h3']) + biases['b3'])
    out_layer = tf.matmul(layer_3, weights['out']) + biases['out']

    return out_layer


def calc(x, y):
    # Returns predictions and error
    predictions = multilayer_NN(x)

    # Define loss and optimizer
    r, c = y.get_shape().as_list()
    r = opts.sgd_batch_size
    print('r:', type(r), r, ',c:', type(c), c)

    # TRAIN: matlab
    # cost1 = 0.5 * sum(sum((pred_real - label_real). ^ 2)) / num_sample;
    # cost2 = 0.5 * sum(sum((pred_imag - label_imag). ^ 2)) / num_sample;
    # cost = cost1 + cost2;
    cost1 = tf.reduce_sum(tf.squared_difference(y[:, :c//2], predictions[:, :c//2]))
    cost2 = tf.reduce_sum(tf.squared_difference(y[:, c//2:], predictions[:, c//2:]))
    loss_t = 0.5*(cost1+cost2)/r
    # mse = tf.losses.mean_squared_error(labels=y, predictions=predictions,weights=0.5)
    # loss_t = tf.divide(tf.reduce_sum(mse), r)


    # DEV: matlab
    # dev_perfs = -mean(sum((dev_label_real - dev_netout1). ^ 2)) - mean(sum((dev_label_imag - dev_netout2). ^ 2));
    mse_r = tf.reduce_sum(tf.squared_difference(y[:, :c // 2], predictions[:, :c // 2]), axis=0)
    mse_i = tf.reduce_sum(tf.squared_difference(y[:, c // 2:], predictions[:, c // 2:]), axis=0)

    loss_d = -tf.reduce_mean(mse_r)-tf.reduce_mean(mse_i)


    return [predictions, loss_t, mse_r, mse_i, loss_d ]

In [ ]:
def write_file(best_weights, best_biases, DNN_NET_FILE):
    W_1, W_2, W_3, W_4 = [np.array(best_weights['h1'], ndmin=2), np.array(best_weights['h2'], ndmin=2),
                          np.array(best_weights['h3'], ndmin=2), np.array([], ndmin=2)]
    W_1, W_2, W_3, W_4 = W_1.T, W_2.T, W_3.T, W_4

    print('W_1: ', W_1.shape, 'W_2: ', W_2.shape, 'W_3: ', W_3.shape, 'W_4: ', W_4.shape)

    b_1, b_2, b_3, b_4 = [np.array(best_biases['b1'], ndmin=2), np.array(best_biases['b2'], ndmin=2),
                          np.array(best_biases['b3'], ndmin=2), np.array([], ndmin=2)]
    b_1, b_2, b_3, b_4 = b_1.T, b_2.T, b_3.T, b_4

    print('b_1: ', b_1.shape, 'b_2: ', b_2.shape, 'b_3: ', b_3.shape, 'b_4: ', b_4.shape)

    Wo, bo = np.array(best_weights['out'], ndmin=2), np.array(best_biases['out'], ndmin=2)

    Wo1_1, Wo1_2, Wo1_3, Wo1_4 = [np.array([], ndmin=2), np.array([], ndmin=2), np.array([], ndmin=2), Wo[:, :963].T]
    bo1_1, bo1_2, bo1_3, bo1_4 = [np.array([], ndmin=2), np.array([], ndmin=2), np.array([], ndmin=2),
                                  bo[:, :963].T]
    # Wo1_1, Wo1_2, Wo1_3, Wo1_4 = [np.array([], ndmin=2), np.array([], ndmin=2), np.array([], ndmin=2), Wo[:963]]
    # bo1_1, bo1_2, bo1_3, bo1_4 = [np.array([],ndmin=2), np.array([],ndmin=2), np.array([],ndmin=2),
    #                               np.reshape(np.transpose(bo[:963]), (963, 1))]


    Wo2_1, Wo2_2, Wo2_3, Wo2_4 = [np.array([], ndmin=2), np.array([], ndmin=2), np.array([], ndmin=2), Wo[:, 963:].T]
    bo2_1, bo2_2, bo2_3, bo2_4 = [np.array([], ndmin=2), np.array([], ndmin=2), np.array([], ndmin=2),
                                  bo[:, 963:].T]
    # Wo2_1, Wo2_2, Wo2_3, Wo2_4 = [np.array([],ndmin=2), np.array([],ndmin=2), np.array([],ndmin=2), Wo[963:]]
    # bo2_1, bo2_2, bo2_3, bo2_4 = [np.array([],ndmin=2), np.array([],ndmin=2), np.array([],ndmin=2),
    #                               np.reshape(np.transpose(bo[963:]), (963, 1))]

    print('Wo1_1: ', Wo1_1.shape, 'Wo1_2: ', Wo1_2.shape, 'Wo1_3: ', Wo1_3.shape, 'Wo1_4: ', Wo1_4.shape)
    print('bo1_1: ', bo1_1.shape, 'bo1_2: ', bo1_2.shape, 'bo1_3: ', bo1_3.shape, 'bo1_4: ', bo1_4.shape)

    print('Wo2_1: ', Wo2_1.shape, 'Wo2_2: ', Wo2_2.shape, 'Wo2_3: ', Wo2_3.shape, 'Wo2_4: ', Wo2_4.shape)
    print('bo2_1: ', bo2_1.shape, 'bo2_2: ', bo2_2.shape, 'bo2_3: ', bo2_3.shape, 'bo2_4: ', bo2_4.shape)

    # Param_Dict = {'W': np.transpose([W_1,W_2, W_3]), 'b':np.transpose([b_1,b_2, b_3]) }
    # Param_Dict = {'W': { (W_1, W_2, (W_3)}, 'b': {(b_1), (b_2), (b_3)}}
    Param_Dict = np.core.records.fromarrays(
        [[W_1, W_2, W_3, W_4], [b_1, b_2, b_3, b_4], [Wo1_1, Wo1_2, Wo1_3, Wo1_4], [bo1_1, bo1_2, bo1_3, bo1_4],
         [Wo2_1, Wo2_2, Wo2_3, Wo2_4], [bo2_1, bo2_2, bo2_3, bo2_4]], names='W,b,Wo1,bo1,Wo2,bo2')

    # print(Param_Dict.shape, Param_Dict)
    master_dict = {'struct_net': [Param_Dict]}

    sio.savemat(DNN_NET_FILE, master_dict, format='5', long_field_names=True)

### main()

In [ ]:
# TRAIN ops
y_p, loss_op, m_r, m_i, _ = calc(X, Y)
train_op = tf.train.AdamOptimizer().minimize(loss=loss_op)


sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Training cycle

for epoch in range(opts.sgd_max_epoch):

    s = time.time()
    cost_sum = 0.0

    ##### Create Train Batch, evaluate COST, Weight, Bias #####
    ###########################################################
    for batch_num, (batch_x, batch_y) in enumerate(opts.next_batch(opts.trData.shape[0], opts.sgd_batch_size)):

        _, c, epoch_w, epoch_b  = sess.run([train_op, loss_op, weights, biases ],
                                          feed_dict={X: batch_x, Y: batch_y})
        cost_sum += c

        if batch_num % 1000 == 0:
            print('[T] - Epoch:', epoch, ', batch_num:', batch_num, ", Cost:", c, "Cost Sum:", cost_sum)

    print('[T] - Epoch:', epoch, ",Sum:", cost_sum)


    ################ Validation in whole batch ################
    ###########################################################

    avg_cost, sum_mse_r, sum_mse_i = 0.0, np.zeros(n_classes//2), np.zeros(n_classes//2)
    ##### Create DEV Batch, evaluate COST, Weight, Bias #####
    ###########################################################
    for batch_num, (batch_x, batch_y) in enumerate(opts.next_batch(opts.cvData.shape[0], opts.sgd_batch_size, isTrainCycle=False)):

        mse_r, mse_i = sess.run([m_r, m_i], feed_dict={X: batch_x, Y: batch_y})
        # print(sum_mse_r.shape, mse_r.shape, mse_i.shape)
        sum_mse_r += (mse_r)
        sum_mse_i += (mse_i)

    avg_cost = - np.mean(sum_mse_r) - np.mean(sum_mse_i)
    # print('[D] - Epoch:', epoch, ", Mean Real:", - np.mean(sum_mse_r), ", Mean Img:" , - np.mean(sum_mse_i), ", Avg Cost:", avg_cost)

    print("[D] - Epoch:{0}, Avg Cost:{1:.5f}, Real(mean,var):({2:.5f},{3:.5f}), Img(mean,var):({4:.5f},{5:.5f})".format( epoch,avg_cost, -np.mean(sum_mse_r),-np.var(sum_mse_r), -np.mean(sum_mse_i),
          -np.var(sum_mse_i)) )

    ####### Min validation error, update weights, bias #########
    ############################################################


    if avg_cost > Best_Cost:
        Best_Cost = avg_cost
        Best_Weight = epoch_w
        Best_Bias = epoch_b
        Best_epoch = epoch

        print('***** [D] - Best Model at Epoch:', epoch, ", Avg Cost:", avg_cost, '*****')


    ######################  Write Model File ###################
    ############################################################

    print('[] - Elapsed Time: {0:.2f}(s), {1:.2f}(min)'.format( time.time()-s, (time.time()-s)/60 ) )

In [ ]:
# Training DONE !!!
print("Optimization Finished!")
print('***** [-] - Best Model at Epoch:', Best_epoch, ", Best Val Cost:", Best_Cost, '*****')
write_file(Best_Weight, Best_Bias, DNN_NET_FILE)
print(" File Write complete")
